In [22]:
import numpy as np
import pandas as pd
from csv import reader
import math
from math import floor
import random
from collections import defaultdict
import itertools

In [5]:
!ls

closest.py		iris.ipynb   Untitled1.ipynb  vectors.py
dlbcl_preprocessed.txt	__pycache__  Untitled2.ipynb
iris-dataset.csv	README.md    Untitled.ipynb


In [26]:
with open('dlbcl_preprocessed.txt', newline='') as db:
    data = list(reader(db, delimiter=' '))

In [19]:
#test

pp = []
for x in data[0][1:] :
    pp.append(float(x))

In [21]:
#test

len(pp)

77

In [44]:
len(data)

2648

In [23]:
#Returns a Vector from a list

class Vector(object):

    def __init__(self, components):
        if not isinstance(components, tuple) and not isinstance(components, list):
            raise ValueError("Components must be a tuple or list.")
        if len(components) == 0:
            raise ValueError("Components must be at least 1-dimensional.")
        self.dimensions = len(components)
        self.components = components

    def __getitem__(self, key):
        return self.components[key]

    def __str__(self):
        return self.components.__str__()

    def __len__(self):
        return self.dimensions

In [24]:
# Measures square of euclidian distance for a nD vector

def cheap_dist(v1, v2):
    comps1 = v1.components
    comps2 = v2.components
    max_dimensions = max(v1.dimensions, v2.dimensions)
    dist = 0.0
    for i in range(0, max_dimensions):
        dist += math.pow(v1[i]-v2[i] , 2)
    return dist

In [25]:
# Measures euclidian distance for a nD vector

def dist(v1, v2):
    return math.sqrt(cheap_dist(v1,v2))

In [33]:
#prepare the dataset

dataset = []
for i in range(len(data)):
    temp = []
    for x in data[i][1:]:
        temp.append(float(x))
    vec = Vector(temp)
    dataset.append(vec)

In [28]:
len(dataset)

2648

In [32]:
len(dataset[500])

77

In [31]:
dataset[0].components

[766.641437876155,
 773.865121153866,
 1405.57817931367,
 998.368320636513,
 1196.41036796559,
 877.489565563797,
 985.997191139054,
 797.388033374492,
 1325.26820413436,
 1384.82630574008,
 539.306852819903,
 997.022693642144,
 1112.144792801,
 963.666662443958,
 942.683469135425,
 800.883724097366,
 706.682697916195,
 759.055352604078,
 1058.60667487167,
 900.632105415155,
 837.664437873338,
 1140.26521632098,
 957.09749281082,
 956.103611243227,
 848.932452103465,
 773.566418876966,
 1425.31125120256,
 693.084990545558,
 1352.71599002224,
 894.452114646232,
 980.131085801716,
 968.549246714021,
 869.053325934099,
 1055.55462230043,
 1143.5619044791,
 957.411288317887,
 992.661930760229,
 1107.60911637902,
 1161.56808898379,
 866.115417823545,
 685.920514560854,
 902.247952120198,
 981.48509264972,
 1059.13842064053,
 937.389765775456,
 1424.54557950785,
 1085.64714855406,
 656.667370272623,
 1023.16908305262,
 790.604641570715,
 668.799688646715,
 850.298999973666,
 784.786042104293

In [48]:



def dist_matrix(points) : 
    n  = len(points)
    max_dist = 0.0
    min_dist = 10000000
    dist_mat = [[-1 for j in range(n)] for i in range(n)]
    for i in range(len(dist_mat)):
        for j in range (i+1,len(dist_mat)):
            dist_mat[i][j] = dist(points[i],points[j])
            if dist_mat[i][j] >= max_dist and dist_mat[i][j] > 0.0 :
                max_dist = dist_mat[i][j]
            elif dist_mat[i][j] <= min_dist :
                min_dist = dist_mat[i][j]
                
    return dist_mat,max_dist,min_dist
    
    

In [49]:
%%time
matrix, max_dist, min_dist = dist_matrix(dataset)

CPU times: user 2min 28s, sys: 72 ms, total: 2min 28s
Wall time: 2min 28s


In [50]:
max_dist

138495.80726704744

In [40]:
min_dist

399.30747850449774

In [47]:
print("Number of genes = {}".format(len(matrix)))
print("Number of features = {}".format(len(dataset[0])))

Number of genes = 2648
Number of features = 77


In [56]:
alpha = 15
th_dist = (max_dist - min_dist)/alpha

In [57]:
th_dist

9206.433319236197

In [59]:
%%time

tot_clu = []
for i in range(len(matrix)) :
    point_clu = []
    for j in range(i+1, len(matrix)) :
        if matrix[i][j] < th_dist :
            point_clu.append(dataset[j])
    tot_clu.append(point_clu)
    

CPU times: user 666 ms, sys: 4 ms, total: 670 ms
Wall time: 670 ms


In [84]:
len(list(set(tot_clu[500]).intersection(tot_clu[600])))


1001

In [85]:
len(list(set(tot_clu[500]) & set(tot_clu[600])))

1001

In [87]:
len(tot_clu[0])

1602

In [88]:
len(tot_clu[1])

1370

In [116]:
# Modified QuickSort
# Given a set of clustures, sorts according to the length of clustures


def partition(arr,low,high): 
    i = ( low-1 )         # index of smaller element 
    pivot = arr[high]     # pivot 
  
    for j in range(low , high): 
  
        # If current element is smaller than or 
        # equal to pivot 
        if   len(arr[j]) <= len(pivot) : 
          
            # increment index of smaller element 
            i = i+1 
            arr[i],arr[j] = arr[j],arr[i] 
  
    arr[i+1],arr[high] = arr[high],arr[i+1] 
    return ( i+1 ) 
  
# The main function that implements QuickSort 
# arr[] --> Array to be sorted, 
# low  --> Starting index, 
# high  --> Ending index 
  
# Function to do Quick sort 
def quickSort(arr,low,high): 
    if low < high: 
  
        # pi is partitioning index, arr[p] is now 
        # at right place 
        pi = partition(arr,low,high) 
  
        # Separately sort elements before 
        # partition and after partition 
        quickSort(arr, low, pi-1) 
        quickSort(arr, pi+1, high) 

In [117]:
test = tot_clu

In [118]:
%%time

n = len(test) 
quickSort(test,0,n-1) 

CPU times: user 148 ms, sys: 1 µs, total: 148 ms
Wall time: 147 ms


In [119]:
test.reverse()

In [123]:
len(test[3])

1685

In [124]:
len(test)

2648

In [125]:
test[-1]

[]

In [133]:
# removes the empty sets

list2 = [x for x in test if x]

In [134]:
len(list2)

2202

In [135]:
list2[-1]

In [137]:
#extent of similarity between two sets

simi_th = 0.9

In [139]:
clu = []
clu.append(list2[0])
for i in range(1,len(list2)):
    clu_new = list2[i]
    common_elements = list(set(clu[-1]) & set(clu_new))
    simi = len(common_elements)/len(clu_new)
    if simi < simi_th :
        clu.append(clu_new)
    
    

In [140]:
len(clu)

1605